# Using the pluto MutualFundsIndia data-set

In [1]:
import pandas as pd
from sqlalchemy import func, or_, and_
from plutoPy.model import RoutingSession, MutualFundsIndia

ModuleNotFoundError: No module named 'Config'

### who are the biggest 5 mutual fund managers?

In [ ]:
lastAumDate = RoutingSession.session.query(func.max(MutualFundsIndia.AumFundwise.PERIOD)).scalar()
print(f"{lastAumDate}")

aums = (RoutingSession.session.query(MutualFundsIndia.AumFundwise.FUND, MutualFundsIndia.AumFundwise.AVG_AUM_WO_FOFD + MutualFundsIndia.AumFundwise.AVG_AUM_FOFD)
        .filter(MutualFundsIndia.AumFundwise.PERIOD == lastAumDate)
        .order_by((MutualFundsIndia.AumFundwise.AVG_AUM_WO_FOFD + MutualFundsIndia.AumFundwise.AVG_AUM_FOFD).desc())).all()

aumDf = pd.DataFrame(aums, columns=['FUND', 'AUM']) #analyze this!
print(aumDf[0:5])